# Rubrica
Que el script no contenga errores
Que los resultados del script sean reproducibles, ie que puedan ser reproducibles en cualquier computadora
El orden del código
La lógica del código
La inferencia empleada para los parámetros utilizados, tales como thresholds para filtrado de matriz, elección de thresholds para selección de alguna banda específica (alfa, beta, delta, etc.)
Los métodos utilizados, ie, si para construir la matriz de conectividad se utilizó coherencia espectral, PLV o correlación, que haya una razón de ello
Los resultados mostrados, tales como mapas de calor, grafos 2D, grafos 3D, el mostrar comunidades, hubs, nodos proporcionales al grado, filtrado de matriz de
Conclusión del análisis, ie, al finalizar el análisis dar una conclusión que resuma los resultados encontrados con una interpretación de los mismos
Que el análisis haya sido aplicado a cada uno de los ensayos/archivos del dataset

Graciela Montserrat Rodríguez Rodríguez
Proyecto, Modelos Computacionales I

Paper:An EEG Dataset of Neural 
Signatures in a Competitive 
two-Player Game Encouraging 
Deceptive Behavior

In [25]:
import numpy as np
import pandas as pd
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as sp
from scipy.io import loadmat
import mne
import glob
import os

In [27]:
#Abrir datos
ruta = r"C:\Datasets\Proyecto_Mon\Proyecto\Preprocessed\DecisionMaking\*.mat"
mat_files = {}

for archivo in glob.glob(ruta):
    nombre = os.path.basename(archivo)
    contenido = loadmat(archivo)
    mat_files[nombre] = contenido


In [28]:
#Codigo para analizar los archivos
if mat_files: 
    first_file_name = list(mat_files.keys())[0]
    data = mat_files[first_file_name] 

    print(f"\n--- Explorando '{first_file_name}' ---")
    print("Nivel 1 (contenido del archivo):")
    print(f"  Claves: {data.keys()}")
    if 'observer' in data:
        observer_content = data['observer']
        print(f"\nNivel 2 (contenido de 'observer'):")
        print(f"  Tipo: {type(observer_content)}")
        print(f"  Shape: {observer_content.shape}")

   
        if observer_content.shape == (1, 1):
            level3_observer = observer_content[0, 0]
            print(f"\nNivel 3 (contenido de 'observer[0,0]'):")
            if hasattr(level3_observer, 'dtype') and level3_observer.dtype.names:
                print(f"  Campos/Claves: {level3_observer.dtype.names}")

                if level3_observer.dtype.names:
                    first_field_name = level3_observer.dtype.names[0]
                    level4_observer = level3_observer[first_field_name]
                    print(f"\nNivel 4 (contenido de '{first_field_name}' dentro de observer):")
                    print(f"  Tipo: {type(level4_observer)}")
                    if hasattr(level4_observer, 'shape'):
                        print(f"  Shape: {level4_observer.shape}")
                     
                    else:
                        print(f"  Contenido: {level4_observer}")
            else:
                print(f"  No hay campos nombrados. Tipo: {type(level3_observer)}, Shape: {level3_observer.shape if hasattr(level3_observer, 'shape') else 'N/A'}")
        else:
            print(f"  No es una estructura 1x1, Shape: {observer_content.shape}")
    else:
        print("La clave 'observer' no se encontró en el archivo.")




--- Explorando 'Player_sub01_Observer_sub02.mat' ---
Nivel 1 (contenido del archivo):
  Claves: dict_keys(['__header__', '__version__', '__globals__', 'observer', 'player'])

Nivel 2 (contenido de 'observer'):
  Tipo: <class 'numpy.ndarray'>
  Shape: (1, 1)

Nivel 3 (contenido de 'observer[0,0]'):
  Campos/Claves: ('fs', 'x', 'clab', 'y', 'className', 't', 'filename')

Nivel 4 (contenido de 'fs' dentro de observer):
  Tipo: <class 'numpy.ndarray'>
  Shape: (1, 1)


In [29]:
#Código para inspeccionar los campos, es decir, encontrar en donde estan los eeg

if mat_files:
    first_file_name = list(mat_files.keys())[0]
    first_file_content = mat_files[first_file_name]

    print(f"\n--- INSPECCIÓN DETALLADA DE '{first_file_name}' ---")

   
    expected_fields = ('fs', 'x', 'clab', 'y', 'className', 't', 'filename')

    print("\n##### Contenido de 'observer' #####")
    if 'observer' in first_file_content and first_file_content['observer'].shape == (1, 1):
        actual_observer_content = first_file_content['observer'][0, 0]

        if hasattr(actual_observer_content, 'dtype') and actual_observer_content.dtype.names:
            print(f"Campos disponibles en 'observer[0,0]': {actual_observer_content.dtype.names}")

            for field_name in expected_fields:
                if field_name in actual_observer_content.dtype.names:
                    field_value = actual_observer_content[field_name]
                    print(f"  --- Campo: '{field_name}' ---")
                    print(f"    Tipo: {type(field_value)}")
                    if hasattr(field_value, 'shape'):
                        print(f"    Shape: {field_value.shape}")
                    
                    if field_name == 'clab' and hasattr(field_value, 'flatten'):
                        
                        print(f"    Primeros 3 elementos (ejemplo): {field_value.flatten()[:3]}")
                    elif field_value is not None:
                        
                        if np.isscalar(field_value) or (isinstance(field_value, np.ndarray) and field_value.size == 1):
                             print(f"    Valor: {field_value}")
                        elif isinstance(field_value, np.ndarray) and field_value.size > 1:
                            
                            print(f"    Primeros 3 elementos (ejemplo): {field_value.flatten()[:3]}")
                else:
                    print(f"  --- Campo: '{field_name}' (No encontrado en observer[0,0]) ---")
        else:
            print(f"Contenido de 'observer[0,0]' no es una estructura con nombres de campo. Tipo: {type(actual_observer_content)}")
    else:
        print("'observer' no encontrado o no es una estructura (1,1).")


    print("\n##### Contenido de 'player' #####")
    if 'player' in first_file_content and first_file_content['player'].shape == (1, 1):
        actual_player_content = first_file_content['player'][0, 0]

        if hasattr(actual_player_content, 'dtype') and actual_player_content.dtype.names:
            print(f"Campos disponibles en 'player[0,0]': {actual_player_content.dtype.names}")

            for field_name in expected_fields:
                if field_name in actual_player_content.dtype.names:
                    field_value = actual_player_content[field_name]
                    print(f"  --- Campo: '{field_name}' ---")
                    print(f"    Tipo: {type(field_value)}")
                    if hasattr(field_value, 'shape'):
                        print(f"    Shape: {field_value.shape}")
                    if field_name == 'clab' and hasattr(field_value, 'flatten'):
                        print(f"    Primeros 3 elementos (ejemplo): {field_value.flatten()[:3]}")
                    elif field_value is not None:
                        if np.isscalar(field_value) or (isinstance(field_value, np.ndarray) and field_value.size == 1):
                             print(f"    Valor: {field_value}")
                        elif isinstance(field_value, np.ndarray) and field_value.size > 1:
                            print(f"    Primeros 3 elementos (ejemplo): {field_value.flatten()[:3]}")
                else:
                    print(f"  --- Campo: '{field_name}' (No encontrado en player[0,0]) ---")
        else:
            print(f"Contenido de 'player[0,0]' no es una estructura con nombres de campo. Tipo: {type(actual_player_content)}")
    else:
        print("'player' no encontrado o no es una estructura (1,1).")


--- INSPECCIÓN DETALLADA DE 'Player_sub01_Observer_sub02.mat' ---

##### Contenido de 'observer' #####
Campos disponibles en 'observer[0,0]': ('fs', 'x', 'clab', 'y', 'className', 't', 'filename')
  --- Campo: 'fs' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (1, 1)
    Valor: [[100]]
  --- Campo: 'x' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (350, 30, 484)
    Primeros 3 elementos (ejemplo): [11.73809814 -4.60339928  4.1760745 ]
  --- Campo: 'clab' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (1, 30)
    Primeros 3 elementos (ejemplo): [array(['F7'], dtype='<U2') array(['F3'], dtype='<U2')
 array(['Fz'], dtype='<U2')]
  --- Campo: 'y' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (4, 484)
    Primeros 3 elementos (ejemplo): [0 0 1]
  --- Campo: 'className' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (1, 4)
    Primeros 3 elementos (ejemplo): [array(['sponL'], dtype='<U5') array(['sponT'], dtype='<U5')
 array(['instL'], dtype='<U5')]
  --- Campo: 't' ---
    Tip

In [31]:
#Matrices de correlación
correlation_matrices_observer = {}
correlation_matrices_player = {}
channel_labels = [] 

if mat_files:
    print("\nMatrices de correlación del jugador y su respectivo jugador...")
    for file_name, file_content in mat_files.items():
        try:
            observer_data_struct = file_content['observer'][0, 0]
            raw_clab = observer_data_struct['clab'][0] 
            channel_labels = [str(item[0]) for item in raw_clab]
            eeg_observer_data_3d = observer_data_struct['x'] #(puntos_de_tiempo, canales, épocas)

            # Reestructurar los datos para hacerla de 2 dimensiones y usar np.corrcoef: (canales x (puntos_de_tiempo * épocas))
            eeg_observer_reshaped = np.transpose(eeg_observer_data_3d, (1, 0, 2))
            # Aplanar los puntos en el tiempo y épocas en una sola dimensión para las muestras
            num_channels_obs = eeg_observer_reshaped.shape[0]
            num_samples_per_channel_obs = eeg_observer_reshaped.shape[1] * eeg_observer_reshaped.shape[2]
            eeg_observer_2d = eeg_observer_reshaped.reshape(num_channels_obs, num_samples_per_channel_obs)
            

            # Calcular la matriz de correlación para el observador
            corr_matrix_observer = np.corrcoef(eeg_observer_2d)
            correlation_matrices_observer[file_name] = corr_matrix_observer

            # PLAYER
            player_data_struct = file_content['player'][0, 0]
            eeg_player_data_3d = player_data_struct['x'] 

           
            eeg_player_reshaped = np.transpose(eeg_player_data_3d, (1, 0, 2))
            num_channels_play = eeg_player_reshaped.shape[0]
            num_samples_per_channel_play = eeg_player_reshaped.shape[1] * eeg_player_reshaped.shape[2]
            eeg_player_2d = eeg_player_reshaped.reshape(num_channels_play, num_samples_per_channel_play)

           
            corr_matrix_player = np.corrcoef(eeg_player_2d)
            correlation_matrices_player[file_name] = corr_matrix_player

            print(f"Calculadas matrices de correlación para '{file_name}' (observador y jugador). "
                  f"Shape de datos EEG observador (original 3D): {eeg_observer_data_3d.shape}, "
                  f"Shape 2D para correlación: {eeg_observer_2d.shape}")

        except KeyError as e:
            print(f"Error: La clave {e} no se encontró en el archivo {file_name}. ¿Estructura inesperada?")
        except Exception as e:
            print(f"Ocurrió un error inesperado al procesar {file_name}: {e}")

    print("\n--- Matrices de correlación calculadas para todos ---")


Matrices de correlación del jugador y su respectivo jugador...
Calculadas matrices de correlación para 'Player_sub01_Observer_sub02.mat' (observador y jugador). Shape de datos EEG observador (original 3D): (350, 30, 484), Shape 2D para correlación: (30, 169400)
Calculadas matrices de correlación para 'Player_sub03_Observer_sub06.mat' (observador y jugador). Shape de datos EEG observador (original 3D): (350, 30, 484), Shape 2D para correlación: (30, 169400)
Calculadas matrices de correlación para 'Player_sub04_Observer_sub05.mat' (observador y jugador). Shape de datos EEG observador (original 3D): (350, 30, 484), Shape 2D para correlación: (30, 169400)
Calculadas matrices de correlación para 'Player_sub05_Observer_sub04.mat' (observador y jugador). Shape de datos EEG observador (original 3D): (350, 30, 484), Shape 2D para correlación: (30, 169400)
Calculadas matrices de correlación para 'Player_sub06_Observer_sub03.mat' (observador y jugador). Shape de datos EEG observador (original 3D

In [40]:
print("\n##### VISUALIZACIÓN DE TODAS LAS MATRICES DE CORRELACIÓN #####")

    # Mostrar DataFrames para el OBSERVADOR
print("\n--- Matrices de Correlación para OBSERVADORES ---")
for file_name, corr_matrix in correlation_matrices_observer.items():
    print(f"\n**{file_name} (OBSERVADOR)**")
    np.fill_diagonal(corr_matrix, 0) # Eliminar diagonales
    p=np.mean(corr_matrix)
    m=np.max(corr_matrix)
    mn=np.min(corr_matrix)
    print(f" El promedio de la matriz de adyacencia es={p} | max={m} | min={mn}")
    df_obs = pd.DataFrame(corr_matrix)
    if channel_labels:
            df_obs.columns = channel_labels
            df_obs.index = channel_labels
    display(df_obs)

    # Mostrar DataFrames para el PLAYER
print("\n--- Matrices de Correlación para PLAYERS ---")
for file_name, corr_matrix in correlation_matrices_player.items():
    print(f"\n**{file_name} (PLAYER)**") # Encabezado para cada DataFrame
    np.fill_diagonal(corr_matrix, 0) # Eliminar diagonales
    p=np.mean(corr_matrix)
    m=np.max(corr_matrix)
    mn=np.min(corr_matrix)
    print(f" El promedio de la matriz de adyacencia es={p} | max={m} | min={mn}")
    df_play = pd.DataFrame(corr_matrix)
    if channel_labels:
            df_play.columns = channel_labels
            df_play.index = channel_labels
    display(df_play) 


##### VISUALIZACIÓN DE TODAS LAS MATRICES DE CORRELACIÓN #####

--- Matrices de Correlación para OBSERVADORES ---

**Player_sub01_Observer_sub02.mat (OBSERVADOR)**
 El promedio de la matriz de adyacencia es=-0.021120990239146896 | max=0.9067388687494867 | min=-0.8377722999378253


,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,O1,PO4,O2,F9,F10,Fp2
F7,0.000000,0.552914,0.178658,0.122604,0.377162,0.639436,0.106931,-0.107348,0.011184,0.420364,...,-0.629567,-0.692986,-0.343619,-0.495286,-0.644251,-0.367424,-0.437591,0.787418,0.559276,0.399234
F3,0.552914,0.000000,0.681477,0.395989,0.095469,0.735887,0.724795,0.403692,0.092363,0.129779,...,-0.300630,-0.522949,-0.563534,-0.536043,-0.632684,-0.546440,-0.599741,0.235729,0.002715,0.284553
Fz,0.178658,0.681477,0.000000,0.722641,0.231484,0.327938,0.773997,0.807485,0.401233,-0.188203,...,-0.145477,-0.308416,-0.543916,-0.620018,-0.556237,-0.658635,-0.642977,-0.090860,-0.026450,0.417330
F4,0.122604,0.395989,0.722641,0.000000,0.545717,0.080637,0.435403,0.775098,0.720659,-0.252658,...,-0.289719,-0.255887,-0.384623,-0.709358,-0.525940,-0.676753,-0.598115,-0.003552,0.224830,0.626695
F8,0.377162,0.095469,0.231484,0.545717,0.000000,-0.004146,-0.100252,0.213438,0.637494,-0.100001,...,-0.553294,-0.366663,-0.137969,-0.656015,-0.497623,-0.512433,-0.434524,0.511482,0.737322,0.752573
FC5,0.639436,0.735887,0.327938,0.080637,-0.004146,0.000000,0.477215,0.053882,-0.145656,0.444236,...,-0.325309,-0.591755,-0.514529,-0.326475,-0.580733,-0.333439,-0.471497,0.399675,-0.001499,0.219618
FC1,0.106931,0.724795,0.773997,0.435403,-0.100252,0.477215,0.000000,0.680619,0.125040,-0.121568,...,0.084255,-0.226402,-0.596073,-0.403303,-0.448088,-0.533018,-0.575640,-0.240030,-0.331319,0.123537
FC2,-0.107348,0.403692,0.807485,0.775098,0.213438,0.053882,0.680619,0.000000,0.557806,-0.373645,...,0.080089,-0.012558,-0.456018,-0.560200,-0.372228,-0.647662,-0.583680,-0.359008,-0.133803,0.330915
FC6,0.011184,0.092363,0.401233,0.720659,0.637494,-0.145656,0.125040,0.557806,0.000000,-0.326314,...,-0.235191,-0.032459,-0.130619,-0.637471,-0.336947,-0.590854,-0.452930,-0.039630,0.364591,0.571569
T7,0.420364,0.129779,-0.188203,-0.252658,-0.100001,0.444236,-0.121568,-0.373645,-0.326314,0.000000,...,-0.309458,-0.446384,-0.155970,0.036204,-0.273540,0.092512,-0.077764,0.463816,0.034875,0.061487



**Player_sub03_Observer_sub06.mat (OBSERVADOR)**
 El promedio de la matriz de adyacencia es=-0.014201265961266004 | max=0.8987925608819015 | min=-0.5668784895588878


,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,O1,PO4,O2,F9,F10,Fp2
F7,0.000000,0.348288,0.102218,0.003201,0.185229,0.207493,0.116661,-0.128736,-0.017870,0.222756,...,-0.394991,-0.403351,-0.210429,-0.224888,-0.119836,-0.387185,-0.070374,0.095848,0.326497,0.243379
F3,0.348288,0.000000,0.552432,0.136606,0.057069,0.272405,0.589431,0.261078,-0.012114,0.045544,...,-0.258981,-0.406093,-0.290221,-0.249224,-0.094157,-0.420108,-0.071096,-0.017659,0.086754,0.252705
Fz,0.102218,0.552432,0.000000,0.358583,0.076719,0.173191,0.810348,0.745202,0.154974,-0.144882,...,-0.209635,-0.360675,-0.301772,-0.358735,-0.103061,-0.439364,-0.109783,-0.093631,-0.004746,0.480862
F4,0.003201,0.136606,0.358583,0.000000,0.189043,-0.019898,0.202909,0.427489,0.160239,-0.139710,...,-0.220163,-0.152107,-0.101240,-0.305310,-0.105788,-0.227147,-0.127981,-0.024293,0.005508,0.193432
F8,0.185229,0.057069,0.076719,0.189043,0.000000,-0.079205,-0.104437,0.005570,-0.096300,-0.099667,...,-0.355746,-0.146522,0.016719,-0.346006,-0.127812,-0.204009,-0.176794,0.284240,0.249596,0.400876
FC5,0.207493,0.272405,0.173191,-0.019898,-0.079205,0.000000,0.388720,0.014045,0.069847,0.161701,...,-0.205393,-0.234680,-0.138050,-0.160591,-0.230836,-0.265928,-0.179292,-0.368020,-0.157968,-0.006957
FC1,0.116661,0.589431,0.810348,0.202909,-0.104437,0.388720,0.000000,0.636424,-0.004979,-0.037776,...,-0.011669,-0.357465,-0.361674,-0.208048,-0.138883,-0.427530,-0.117030,-0.214270,-0.167246,0.160761
FC2,-0.128736,0.261078,0.745202,0.427489,0.005570,0.014045,0.636424,0.000000,0.406025,-0.265339,...,-0.020153,-0.036396,-0.126920,-0.388825,-0.216864,-0.226383,-0.246759,-0.253429,-0.220281,0.171282
FC6,-0.017870,-0.012114,0.154974,0.160239,-0.096300,0.069847,-0.004979,0.406025,0.000000,-0.151878,...,-0.240007,0.127608,0.126778,-0.389140,-0.276497,-0.045123,-0.331969,-0.167577,-0.316364,0.058050
T7,0.222756,0.045544,-0.144882,-0.139710,-0.099667,0.161701,-0.037776,-0.265339,-0.151878,0.000000,...,-0.254587,-0.296219,-0.185385,-0.121459,-0.102239,-0.269974,-0.061950,0.233742,-0.026405,-0.151976



**Player_sub04_Observer_sub05.mat (OBSERVADOR)**
 El promedio de la matriz de adyacencia es=-0.0061484321382129555 | max=0.9939866882841476 | min=-0.7362810322334105


...

,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,O1,PO4,O2,F9,F10,Fp2
F7,0.000000,0.301302,-0.058059,-0.014546,0.421108,0.580462,-0.121440,-0.314220,0.029539,0.321119,...,-0.664862,-0.577507,-0.294464,-0.471555,-0.220722,-0.546524,-0.408498,0.999422,0.797944,0.348800
F3,0.301302,0.000000,0.667223,0.428127,0.158393,0.708159,0.658186,0.376613,0.188091,-0.166060,...,-0.439366,-0.583744,-0.543389,-0.441682,-0.503355,-0.585623,-0.570731,0.281838,0.117825,0.045914
Fz,-0.058059,0.667223,0.000000,0.675439,0.197540,0.333493,0.880070,0.849928,0.383810,-0.209913,...,-0.290831,-0.542009,-0.589650,-0.496285,-0.667282,-0.566263,-0.633520,-0.079747,-0.066513,0.214716
F4,-0.014546,0.428127,0.675439,0.000000,0.397623,0.175556,0.457007,0.696099,0.669859,-0.241613,...,-0.354060,-0.443489,-0.369887,-0.545603,-0.579611,-0.467525,-0.527890,-0.026531,0.146886,-0.036872
F8,0.421108,0.158393,0.197540,0.397623,0.000000,0.159852,-0.041311,0.138209,0.627405,0.061683,...,-0.610846,-0.521076,-0.128647,-0.691544,-0.494246,-0.521435,-0.497673,0.413845,0.672644,0.546717
FC5,0.580462,0.708159,0.333493,0.175556,0.159852,0.000000,0.384423,0.037671,0.048108,0.047367,...,-0.519461,-0.620354,-0.519680,-0.352898,-0.365030,-0.605207,-0.533148,0.558180,0.238039,0.220794
FC1,-0.121440,0.658186,0.880070,0.457007,-0.041311,0.384423,0.000000,0.759156,0.143693,-0.226180,...,-0.116880,-0.450770,-0.615015,-0.281562,-0.559949,-0.473978,-0.545872,-0.145043,-0.271023,0.108267
FC2,-0.314220,0.376613,0.849928,0.696099,0.138209,0.037671,0.759156,0.000000,0.469559,-0.285427,...,-0.070751,-0.339510,-0.470202,-0.398256,-0.631207,-0.378867,-0.507182,-0.331721,-0.186509,0.166828
FC6,0.029539,0.188091,0.383810,0.669859,0.627405,0.048108,0.143693,0.469559,0.000000,-0.132777,...,-0.427904,-0.401443,-0.152790,-0.621594,-0.551896,-0.420702,-0.467099,0.021424,0.367330,0.367799
T7,0.321119,-0.166060,-0.209913,-0.241613,0.061683,0.047367,-0.226180,-0.285427,-0.132777,0.000000,...,-0.274769,-0.252245,-0.100719,-0.098454,0.028338,-0.230839,-0.147686,0.318283,0.149096,0.447443



**Player_sub23_Observer_sub24.mat (PLAYER)**
 El promedio de la matriz de adyacencia es=-0.024004339435774402 | max=0.9711345881051314 | min=-0.6997119153271644


,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,O1,PO4,O2,F9,F10,Fp2
F7,0.000000,0.574851,0.325109,0.185460,0.334167,0.654954,0.322699,0.161646,0.157565,0.640804,...,-0.519528,-0.622779,-0.448790,-0.471540,-0.453965,-0.560610,-0.470484,0.824568,0.391328,0.112538
F3,0.574851,0.000000,0.674007,0.342391,0.200988,0.584975,0.710960,0.459590,0.134647,0.254500,...,-0.431563,-0.629977,-0.587895,-0.529645,-0.515805,-0.616362,-0.538444,0.385679,0.129803,0.265542
Fz,0.325109,0.674007,0.000000,0.615099,0.313119,0.308366,0.870017,0.792392,0.292955,-0.015428,...,-0.398607,-0.582261,-0.607842,-0.590473,-0.574960,-0.619843,-0.550555,0.152671,0.177708,0.402701
F4,0.185460,0.342391,0.615099,0.000000,0.517261,0.134920,0.475361,0.638695,0.529216,-0.071383,...,-0.410064,-0.449256,-0.415061,-0.564008,-0.546747,-0.524217,-0.472320,0.126695,0.414947,0.404257
F8,0.334167,0.200988,0.313119,0.517261,0.000000,0.160397,0.196649,0.315645,0.774649,0.115040,...,-0.534773,-0.465660,-0.257857,-0.571327,-0.544347,-0.499205,-0.430636,0.479640,0.854015,0.360564
FC5,0.654954,0.584975,0.308366,0.134920,0.160397,0.000000,0.335332,0.147109,0.049386,0.501467,...,-0.397835,-0.541660,-0.459288,-0.371710,-0.365932,-0.508798,-0.444493,0.504607,0.143743,0.201164
FC1,0.322699,0.710960,0.870017,0.475361,0.196649,0.335332,0.000000,0.758768,0.164332,-0.061858,...,-0.260631,-0.589324,-0.699712,-0.573688,-0.566640,-0.648298,-0.589091,0.101696,0.042351,0.506505
FC2,0.161646,0.459590,0.792392,0.638695,0.315645,0.147109,0.758768,0.000000,0.314244,-0.171124,...,-0.262263,-0.447443,-0.566690,-0.574990,-0.565399,-0.559682,-0.520561,0.010335,0.172943,0.445122
FC6,0.157565,0.134647,0.292955,0.529216,0.774649,0.049386,0.164332,0.314244,0.000000,-0.045441,...,-0.422187,-0.324344,-0.210868,-0.508204,-0.491501,-0.414732,-0.375916,0.177241,0.677568,0.453380
T7,0.640804,0.254500,-0.015428,-0.071383,0.115040,0.501467,-0.061858,-0.171124,-0.045441,0.000000,...,-0.386745,-0.425624,-0.247332,-0.151912,-0.154179,-0.362769,-0.316655,0.705545,0.192854,0.132785



**Player_sub24_Observer_sub23.mat (PLAYER)**
 El promedio de la matriz de adyacencia es=-0.02320372452857914 | max=0.9809290657566344 | min=-0.7507153617928047


,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,O1,PO4,O2,F9,F10,Fp2
F7,0.000000,-0.124333,-0.081612,0.082483,0.186987,-0.266367,-0.109568,-0.109394,0.031857,0.209543,...,-0.330804,-0.310896,-0.176981,-0.204638,-0.187970,-0.296337,-0.199821,0.376290,0.299993,0.180065
F3,-0.124333,0.000000,0.781539,0.151634,-0.091049,0.532813,0.885470,0.704979,0.123235,-0.010833,...,-0.204097,-0.541082,-0.523420,-0.520296,-0.537902,-0.555009,-0.485407,-0.233224,-0.216341,0.372292
Fz,-0.081612,0.781539,0.000000,0.406719,0.014209,0.196846,0.920009,0.918477,0.217813,-0.172905,...,-0.159163,-0.510123,-0.547038,-0.558107,-0.589934,-0.533232,-0.508140,-0.306558,-0.269297,0.572130
F4,0.082483,0.151634,0.406719,0.000000,0.326206,-0.082190,0.306469,0.461341,0.392960,-0.097735,...,-0.288544,-0.416142,-0.316635,-0.447657,-0.483126,-0.444395,-0.392373,-0.134056,-0.108536,0.676746
F8,0.186987,-0.091049,0.014209,0.326206,0.000000,-0.048969,-0.058277,0.060711,0.320590,0.027931,...,-0.438986,-0.383422,-0.165920,-0.388491,-0.374206,-0.379257,-0.310062,0.288171,0.191742,0.657520
FC5,-0.266367,0.532813,0.196846,-0.082190,-0.048969,0.000000,0.315855,0.106711,0.029906,0.139322,...,-0.251660,-0.366001,-0.305036,-0.250917,-0.251334,-0.360516,-0.263046,0.062768,0.071620,0.062690
FC1,-0.109568,0.885470,0.920009,0.306469,-0.058277,0.315855,0.000000,0.850516,0.083133,-0.201159,...,-0.043555,-0.471013,-0.585221,-0.510295,-0.551449,-0.494409,-0.485163,-0.299337,-0.301504,0.422275
FC2,-0.109394,0.704979,0.918477,0.461341,0.060711,0.106711,0.850516,0.000000,0.288915,-0.293670,...,-0.068135,-0.440424,-0.533456,-0.571492,-0.605048,-0.473326,-0.498642,-0.343124,-0.275782,0.543368
FC6,0.031857,0.123235,0.217813,0.392960,0.320590,0.029906,0.083133,0.288915,0.000000,-0.076945,...,-0.355327,-0.360723,-0.206045,-0.465034,-0.439264,-0.358402,-0.313394,0.026061,0.083797,0.506115
T7,0.209543,-0.010833,-0.172905,-0.097735,0.027931,0.139322,-0.201159,-0.293670,-0.076945,0.000000,...,-0.320291,-0.252676,-0.121326,0.003393,0.021811,-0.227922,-0.108970,0.359779,0.241026,0.058497


In [23]:
#Umbralización, eliminación de la diágonal, creación de grafos del observador y métricas
Grafos= {}
for file_name, corr_matrix in correlation_matrices_player.items():
    threshold = 0.5
    print(f"Matrix: {key}, Threshold: {threshold:.4f}")
    matrix_data[matrix_data < threshold] = 0

In [3]:
##Grafos y métricas de los observadorres, grafo binario no direccionado
Grafos= {}
for key, corr_matrix in correlation_matrices_observer.items():
    G = nx.from_numpy_array(corr_matrix)
    Grafos[key] = G


for key, corr_matrix in correlation_matrices_observer.items():
        print(f"\n**{file_name} (OBSERVADOR)**") 
        df_obs = pd.DataFrame(corr_matrix)
        if channel_labels:
            df_obs.columns = channel_labels
            df_obs.index = channel_labels
        display(df_obs) 


                            
##https://iopscience.iop.org/article/10.1088/1742-6596/1727/1/012009 umbralización por densidad

NameError: name 'correlation_matrices_observer' is not defined

In [ ]:
##Tipo de grafo: nodos, aristas, conexiones
##Matrices de adyacencia, por condiciones
## En el escrito deben estar los grafos. 3D o 2D, no importa mucho para Oswaldo, para fer  tal vez sí iji
## Metricas: grado, ¿qué tipo de grafo es?, binarizar, y sacar hubs.
## Para sacar umbral: media+desviación estandar
## Coeficiente de cluster y rutas más cortas, apartir de ahí mundo pequeño
## Betweeness, intermediación, cercanía, rich club, comunidades, modularidad, gráficas de escala libre (es decir, que la distribución de grados siga 
## una ley potencia)
## A partir de ahí, hacer gráficas y estadísticas adecuadas, histograma y percentiles

#Bibliografia
Nayak, C. S., & Anilkumar, A. C. (2025, 3 agosto). EEG Normal waveforms. StatPearls - NCBI Bookshelf. https://www.ncbi.nlm.nih.gov/books/NBK539805/